# Access to ECMWF data

Access ECMWF Open Data: https://www.ecmwf.int/en/forecasts/datasets/open-data \
Forecasted variables can be accessed through `ecwmf.opendata`

## Setup

In [28]:
import datetime
import eccodes
from ecmwf.opendata import Client
import pdbufr

Will the user select the timestep from the widget? If so, date/time and step needed to retrieve ecmwf data will have to be provided as variables. Below I'm setting them as tool variables. Date and time can be provided in different styles: https://github.com/ecmwf/ecmwf-opendata/tree/main#date-and-time

In [17]:
# Tool variables
date = '2023-07-30'
time = 0 #available times: 0, 6, 12, 18
step = 48 #hours

## 0. General retrieval process using ecmwf.opendata

```python
client = Client(source="ecmwf")

request = {
    "time": ,
    "stream": ,
    "type": , #type of data, default is fc
    "step": ,
    "date": ,
    "param": , #meteorological parameters. multiple parameters can be specified in a list.
}

client.retrieve(request = request, target = "") #takes request as input and will retrieve the corresponding data from the server and write them in the user's target file
client.download(request = request, target = "") #will download the whole data files from the server, ignoring keywords like param, levelist or number
client.latest(request = request, target = "")   #returns the date of the most recent matching forecast without downloading the data
```

To obtain data to be used in the dashboard, it will have to be retrieved through `Client.retrieve`, then uploaded in the dashboard.

More info on request parameters: https://github.com/ecmwf/ecmwf-opendata/tree/main#request-keywords

Tutoral on GRIB files: https://confluence.ecmwf.int/display/CKB/How+to+plot+GRIB+files+with+Python+and+matplotlib

## 1. Cyclone characteristics
List of available parameters: https://github.com/ecmwf/ecmwf-opendata/tree/main#parameters-and-levels

In [25]:
# Download the Tropical Cyclone tracks from ECMWF's 00UTC ENS forecast
client.retrieve(
    time=0,
    stream="enfo",
    type="tf",
    step=240,
    target="track_data/ens_tracks.bufr",
)

20230727000000-240h-enfo-tf.bufr:   0%|          | 0.00/535k [00:00<?, ?B/s]

In [27]:
# Import bufr file of the TC tracks
df_tracks = pdbufr.read_bufr("track_data/ens_tracks.bufr",
    columns=("stormIdentifier", "ensembleMemberNumber", "latitude", "longitude",
             "pressureReducedToMeanSeaLevel",))
storms = df_tracks.stormIdentifier.unique()

In [4]:
# Import bufr file of the TC track
df = pdbufr.read_bufr("../data/A_JSXX01ECEP250000_C_ECMP_20230725000000_tropical_cyclone_track_DOKSURI_124p6degE_17p6degN_bufr4.bin",
    columns=("stormIdentifier", "ensembleMemberNumber", "latitude", "longitude",
             "pressureReducedToMeanSeaLevel",))
# Save the number of ensembles in a list
members = df.ensembleMemberNumber.unique()
df.head()

,stormIdentifier,ensembleMemberNumber,latitude,longitude,pressureReducedToMeanSeaLevel
0,07W,1,17.7,124.5,96600.0
1,07W,1,18.5,123.9,96100.0
2,07W,1,19.0,122.8,96200.0
3,07W,1,19.4,121.7,95900.0
4,07W,1,19.3,121.0,95500.0


In [3]:
# Cyclone temporal horizon definition
# For DOKSURI forecast of the 25th July we have 41 snapshot with 6h step
start_dte = datetime(2023, 7, 25)
end_dte = datetime(2023, 8, 4)
step = timedelta(hours=6)
time_steps = pd.date_range(start_dte, end_dte, freq=step)

In [4]:
# Save all ensemble members track in a list called locations
locations = []
tracks_len = []
tracks_date = []
for m in members:
    dft = df[df.ensembleMemberNumber == m]
    dft_noNaN = dft.dropna()
    lat = dft_noNaN.latitude.values
    lon = dft_noNaN.longitude.values
    locs = [[lat[i], lon[i]] for i in range(len(lat))]
    track_len = len(locs)
    date_range = time_steps[:track_len]
    locations.append(locs)
    tracks_len.append(track_len)
    tracks_date.append(date_range)

## 2. Atmosphere and ocean characteristics

In [27]:
# Variable: Wind speed
# Download from ECMWF open data
request = {
    "time": time,
    # "stream": "fc", #HRES
    "type": "fc",
    "step": step,
    # "date": date,
    "param": ["u", "v"],
}

client = Client(source = "ecmwf")
client.retrieve(request = request, target = "test_data_access/wind.grib2")

20230730000000-48h-oper-fc.grib2:   0%|          | 0.00/5.11M [00:00<?, ?B/s]

In [29]:
# Open the file and put it in a usable format
f = open("test_data_access/wind.grib2")


In [31]:
f.info()

AttributeError: '_io.TextIOWrapper' object has no attribute 'info'

In [ ]:
# Variable: 
# Download from ECMWF open data
request = {
    "time": time,
    "stream": "fc", #HRES
    
}


In [ ]:
stream = "wave" #ocean characteristics

## 3. Impact variables